# Eurostat energy balance 2018

Extract bioenergy related data from an archive containing XLSB files, one for each EU country which contain multiple sheets for each year (1990-2018).

In [1]:
import os
import zipfile
import requests
import pandas as pd
import numpy as np
import pyxlsb

In [2]:
def get_values(file, country, variable, fuel, col_start, col_end):
    """Reads fuel variable in multiple sheets 2002-2018.
    Sums the values across multiple columns if relevant.
    Returns: dict
    """
    trajectory = {}
    for year in range(2002,2019):
        df = pd.read_excel(
            file,
            engine='pyxlsb',
            sheet_name=str(year),
            skiprows=[0,1,2,3],
            index_col=1
            )
        try:
            trajectory[(country, year, fuel)] = df.loc[variable, col_start:col_end].sum()
        except TypeError:
            trajectory[(country, year, fuel)] = pd.to_numeric(df.loc[variable, col_start:col_end], errors='coerce').sum()
    return trajectory


In [3]:
def read_excels(directory, var, var_key, col_start, col_end):
    d = {}
    
    for filename in os.listdir(directory):
        if '!' not in filename: # skip readme files
            country = filename.split('-')[0]
            excel_path = os.path.join(directory, filename)
            data = get_values(excel_path, country, var, var_key, col_start, col_end)
            d.update(data)
    return d

In [4]:
tj_ktoe = 41.868

In [5]:
url = 'https://ec.europa.eu/eurostat/documents/38154/4956218/Energy-Balances-April-2020-edition.zip/69da6e9f-bf8f-cd8e-f4ad-50b52f8ce616'

r = requests.get(url)

with open('eurostat_balances_2020.zip', 'wb') as f:
    f.write(r.content)

In [6]:
# All variables of interest
all_data = {}

variables = [
    'Primary production',
    'Imports',
    'Exports',
    'Gross inland consumption',
    ]

fuel_tuples = [
    ('total', 'Total', 'Total'),
    ('renewables', 'Renewables and biofuels', 'Renewables and biofuels'),
    ('bioenergy', 'Bioenergy', 'Bioenergy',),
    ('solid_biomass', 'Primary solid biofuels', 'Primary solid biofuels'),
    ('biofuels', 'Pure biogasoline', 'Other liquid biofuels'),
    ('biogas', 'Biogases', 'Biogases'),
    ('ren_mun_waste', 'Renewable municipal waste', 'Renewable municipal waste'),
    ]

with zipfile.ZipFile('eurostat_balances_2020.zip', 'r') as zip_archive:
    zip_archive.extractall(path='balances/')

for variable in variables:
    for fuel_key, start, end in fuel_tuples:
        data = read_excels('balances', variable, fuel_key, start, end)
        all_data.setdefault(variable.lower().replace(' ', '_'), {}).update(data)

df = pd.DataFrame(all_data)
df.index.names = ['country', 'year', 'fuel']

In [7]:
df

primary_production  imports  exports  \
country year fuel                                                 
LT      2002 total                    5227.83  9372.06  5679.77   
        2003 total                    5552.45    10490  6446.45   
        2004 total                    5392.67  12284.9  7911.38   
        2005 total                     4145.6  12800.7  7751.96   
        2006 total                     3737.9  12485.8  7048.76   
...                                       ...      ...      ...   
ES      2014 ren_mun_waste            204.189        0        0   
        2015 ren_mun_waste            252.006        0        0   
        2016 ren_mun_waste            235.239        0        0   
        2017 ren_mun_waste            259.661        0        0   
        2018 ren_mun_waste            254.132        0        0   

                           gross_inland_consumption  
country year fuel                                    
LT      2002 total                          8983.61  
        2003 total                          9369.74  
        2004 total                          9495.61  
        2005 total                          8982.81  
        2006 total                          8872.44  
...                                             ...  
ES      2014 ren_mun_waste                  204.189  
        2015 ren_mun_waste                  252.006  
        2016 ren_mun_waste                  235.239  
        2017 ren_mun_waste                  259.661  
        2018 ren_mun_waste                  254.132  

[5117 rows x 4 columns]

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5117 entries, ('LT', 2002, 'total') to ('ES', 2018, 'ren_mun_waste')
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   primary_production        5117 non-null   object
 1   imports                   5117 non-null   object
 2   exports                   5117 non-null   object
 3   gross_inland_consumption  5117 non-null   object
dtypes: object(4)
memory usage: 177.8+ KB


In [9]:
df.index.unique(level=0)

Index(['LT', 'EE', 'UK', 'IE', 'MK', 'DE', 'MT', 'AT', 'XK', 'BA', 'RO', 'NL',
       'HR', 'UA', 'GE', 'AL', 'SE', 'EU27_2020', 'LV', 'CY', 'MD', 'FR', 'HU',
       'RS', 'FI', 'PT', 'BG', 'ME', 'DK', 'IT', 'TR', 'SI', 'NO', 'EL', 'SK',
       'CZ', 'LU', 'IS', 'EU28', 'PL', 'EA19', 'BE', 'ES'],
      dtype='object', name='country')

In [10]:
df.loc[pd.IndexSlice[:, :, :], :]

primary_production  imports  exports  \
country year fuel                                                 
LT      2002 total                    5227.83  9372.06  5679.77   
        2003 total                    5552.45    10490  6446.45   
        2004 total                    5392.67  12284.9  7911.38   
        2005 total                     4145.6  12800.7  7751.96   
        2006 total                     3737.9  12485.8  7048.76   
...                                       ...      ...      ...   
ES      2014 ren_mun_waste            204.189        0        0   
        2015 ren_mun_waste            252.006        0        0   
        2016 ren_mun_waste            235.239        0        0   
        2017 ren_mun_waste            259.661        0        0   
        2018 ren_mun_waste            254.132        0        0   

                           gross_inland_consumption  
country year fuel                                    
LT      2002 total                          8983.61  
        2003 total                          9369.74  
        2004 total                          9495.61  
        2005 total                          8982.81  
        2006 total                          8872.44  
...                                             ...  
ES      2014 ren_mun_waste                  204.189  
        2015 ren_mun_waste                  252.006  
        2016 ren_mun_waste                  235.239  
        2017 ren_mun_waste                  259.661  
        2018 ren_mun_waste                  254.132  

[5117 rows x 4 columns]

In [11]:
df.to_csv(
    'bioenergy_2002_2018.csv',
    decimal=',',
    )

In [12]:
df2 = df.copy()


In [13]:
df2 = df2.apply(pd.to_numeric, downcast='float', errors='coerce')

In [14]:
df2

primary_production       imports      exports  \
country year fuel                                                           
LT      2002 total                 5227.826172   9372.054688  5679.765137   
        2003 total                 5552.446777  10489.969727  6446.448242   
        2004 total                 5392.669922  12284.943359  7911.381836   
        2005 total                 4145.604004  12800.707031  7751.960938   
        2006 total                 3737.903076  12485.837891  7048.756836   
...                                        ...           ...          ...   
ES      2014 ren_mun_waste          204.188995      0.000000     0.000000   
        2015 ren_mun_waste          252.005997      0.000000     0.000000   
        2016 ren_mun_waste          235.238998      0.000000     0.000000   
        2017 ren_mun_waste          259.661011      0.000000     0.000000   
        2018 ren_mun_waste          254.132004      0.000000     0.000000   

                            gross_inland_consumption  
country year fuel                                     
LT      2002 total                       8983.607422  
        2003 total                       9369.743164  
        2004 total                       9495.610352  
        2005 total                       8982.805664  
        2006 total                       8872.442383  
...                                              ...  
ES      2014 ren_mun_waste                204.188995  
        2015 ren_mun_waste                252.005997  
        2016 ren_mun_waste                235.238998  
        2017 ren_mun_waste                259.661011  
        2018 ren_mun_waste                254.132004  

[5117 rows x 4 columns]

In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5117 entries, ('LT', 2002, 'total') to ('ES', 2018, 'ren_mun_waste')
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   primary_production        4900 non-null   float32
 1   imports                   4900 non-null   float32
 2   exports                   4900 non-null   float32
 3   gross_inland_consumption  4900 non-null   float32
dtypes: float32(4)
memory usage: 97.8+ KB


In [16]:
df2['dependency'] = (df2['imports'] - df2['exports']) / df2['gross_inland_consumption']

In [17]:
df2

primary_production       imports      exports  \
country year fuel                                                           
LT      2002 total                 5227.826172   9372.054688  5679.765137   
        2003 total                 5552.446777  10489.969727  6446.448242   
        2004 total                 5392.669922  12284.943359  7911.381836   
        2005 total                 4145.604004  12800.707031  7751.960938   
        2006 total                 3737.903076  12485.837891  7048.756836   
...                                        ...           ...          ...   
ES      2014 ren_mun_waste          204.188995      0.000000     0.000000   
        2015 ren_mun_waste          252.005997      0.000000     0.000000   
        2016 ren_mun_waste          235.238998      0.000000     0.000000   
        2017 ren_mun_waste          259.661011      0.000000     0.000000   
        2018 ren_mun_waste          254.132004      0.000000     0.000000   

                            gross_inland_consumption  dependency  
country year fuel                                                 
LT      2002 total                       8983.607422    0.411003  
        2003 total                       9369.743164    0.431551  
        2004 total                       9495.610352    0.460588  
        2005 total                       8982.805664    0.562046  
        2006 total                       8872.442383    0.612805  
...                                              ...         ...  
ES      2014 ren_mun_waste                204.188995    0.000000  
        2015 ren_mun_waste                252.005997    0.000000  
        2016 ren_mun_waste                235.238998    0.000000  
        2017 ren_mun_waste                259.661011    0.000000  
        2018 ren_mun_waste                254.132004    0.000000  

[5117 rows x 5 columns]

In [18]:
df2.sort_index(ascending=True, inplace=True)

In [19]:
df2

primary_production     imports    exports  \
country year fuel                                                       
AL      2002 bioenergy              255.994995    0.000000   0.000000   
             biofuels                 0.000000    0.000000   0.000000   
             biogas                   0.000000    0.000000   0.000000   
             ren_mun_waste            0.000000    0.000000   0.000000   
             renewables             559.382019    0.000000   0.000000   
...                                        ...         ...        ...   
XK      2018 biogas                   0.000000    0.000000   0.000000   
             ren_mun_waste            0.000000    0.000000   0.000000   
             renewables             394.709015    9.141000   0.000000   
             solid_biomass          368.170990    9.141000   0.000000   
             total                 1822.274048  826.314026  69.603996   

                            gross_inland_consumption  dependency  
country year fuel                                                 
AL      2002 bioenergy                    255.994995    0.000000  
             biofuels                       0.000000         NaN  
             biogas                         0.000000         NaN  
             ren_mun_waste                  0.000000         NaN  
             renewables                   559.382019    0.000000  
...                                              ...         ...  
XK      2018 biogas                         0.000000         NaN  
             ren_mun_waste                  0.000000         NaN  
             renewables                   403.850006    0.022635  
             solid_biomass                377.312012    0.024227  
             total                       2584.928955    0.292739  

[5117 rows x 5 columns]

In [20]:
df2.to_csv(
    'eurostat_2002_2018_ktoe.csv',
    decimal=',',
    )

In [21]:
df3 = df2.copy()

In [22]:
tj_ktoe = 41.868

df3 = df3.loc[:, 'primary_production': 'gross_inland_consumption'] * tj_ktoe

# Keep the share based on the original data in ktoe
df3['dependency'] = df2['dependency']
df3

primary_production       imports      exports  \
country year fuel                                                           
AL      2002 bioenergy            10717.998456      0.000000     0.000000   
             biofuels                 0.000000      0.000000     0.000000   
             biogas                   0.000000      0.000000     0.000000   
             ren_mun_waste            0.000000      0.000000     0.000000   
             renewables           23420.206373      0.000000     0.000000   
...                                        ...           ...          ...   
XK      2018 biogas                   0.000000      0.000000     0.000000   
             ren_mun_waste            0.000000      0.000000     0.000000   
             renewables           16525.677036    382.715379     0.000000   
             solid_biomass        15414.583009    382.715379     0.000000   
             total                76294.969835  34596.115635  2914.180116   

                            gross_inland_consumption  dependency  
country year fuel                                                 
AL      2002 bioenergy                  10717.998456    0.000000  
             biofuels                       0.000000         NaN  
             biogas                         0.000000         NaN  
             ren_mun_waste                  0.000000         NaN  
             renewables                 23420.206373    0.000000  
...                                              ...         ...  
XK      2018 biogas                         0.000000         NaN  
             ren_mun_waste                  0.000000         NaN  
             renewables                 16908.392056    0.022635  
             solid_biomass              15797.299307    0.024227  
             total                     108225.805491    0.292739  

[5117 rows x 5 columns]

In [23]:
df3.to_csv(
    'eurostat_2002_2018_tj.csv',
    decimal=',',
    )

In [24]:
df2.loc[pd.IndexSlice['CZ', 2002:2018], :]

primary_production       imports      exports  \
country year fuel                                                           
CZ      2002 bioenergy             1768.482056      5.427000    39.639999   
             biofuels                92.261002      0.035000    27.660999   
             biogas                  35.945999      0.000000     0.000000   
             ren_mun_waste           56.008999      0.000000     0.000000   
             renewables            1988.066040      0.035000    27.660999   
...                                        ...           ...          ...   
        2018 biogas                 603.781982      0.000000     0.000000   
             ren_mun_waste           87.627998      0.000000     0.000000   
             renewables            4570.729980    431.136993   428.973999   
             solid_biomass         3070.351074    206.854004   296.377991   
             total                27345.869141  22992.556641  6989.462891   

                            gross_inland_consumption  dependency  
country year fuel                                                 
CZ      2002 bioenergy                   1734.145020   -0.019729  
             biofuels                      64.512001   -0.428230  
             biogas                        35.945999    0.000000  
             ren_mun_waste                 56.008999    0.000000  
             renewables                  1960.317017   -0.014093  
...                                              ...         ...  
        2018 biogas                       603.781982    0.000000  
             ren_mun_waste                 87.627998    0.000000  
             renewables                  4569.796875    0.000473  
             solid_biomass               2980.826904   -0.030033  
             total                      43551.718750    0.367450  

[119 rows x 5 columns]

In [25]:
df3.loc[pd.IndexSlice[['AT', 'CZ', 'DK', 'DE', 'SK', 'PL', 'UK'],2018,'bioenergy'], :]

,,,primary_production,imports,exports,gross_inland_consumption,dependency
country,year,fuel,,,,,
AT,2018,bioenergy,2.240478e+05,46039.809907,36676.912304,2.339790e+05,0.040016
CZ,2018,bioenergy,1.667133e+05,20343.326184,23004.498327,1.639224e+05,-0.016234
DK,2018,bioenergy,1.063762e+05,87731.419491,12966.058858,1.812002e+05,0.412612
DE,2018,bioenergy,1.082730e+06,117757.307145,129846.651258,1.070641e+06,-0.011292
SK,2018,bioenergy,5.182438e+04,7459.998408,7719.873114,5.129508e+04,-0.005066
PL,2018,bioenergy,3.114666e+05,39538.088220,29267.993547,3.210062e+05,0.031993
UK,2018,bioenergy,3.776739e+05,168692.908087,12383.465280,5.334908e+05,0.292994


In [26]:
# Share of bioenegy on consumption
df3.loc[pd.IndexSlice['CZ',2018,'bioenergy'], 'gross_inland_consumption']/df3.loc[pd.IndexSlice['CZ',2018,'total'], 'gross_inland_consumption']

0.08989817355263677

In [27]:
df3.loc[pd.IndexSlice[['AT', 'CZ', 'DK', 'DE', 'SK', 'PL', 'UK'],2018,'bioenergy'], :].to_csv('dependency_2018_bioenergy_selected_tj.csv', decimal=',')

In [28]:
df3.loc[pd.IndexSlice[['AT', 'CZ', 'DK', 'DE', 'SK', 'PL', 'UK'],2018,'solid_biomass'], :].to_csv('dependency_2018_solid_selected_tj.csv', decimal=',')

In [29]:
pd.pivot_table(df3.loc[pd.IndexSlice[['AT', 'CZ', 'DK', 'DE', 'SK', 'PL', 'UK'],2002:2018,'bioenergy'], :], index='year', columns='country', values='dependency')

country,AT,CZ,DE,DK,PL,SK,UK
year,,,,,,,
2002,-0.002344,-0.019729,0.001055,0.080602,-0.000458,-0.006294,0.005929
2003,-0.028737,-0.079935,0.023298,0.086022,-0.000658,-0.016159,0.047902
2004,-0.052183,-0.082543,0.025388,0.126475,-0.001240,-0.043027,0.139927
2005,-0.007057,-0.105124,0.056233,0.176998,-0.017443,-0.099175,0.135144
2006,0.079718,-0.120045,0.053409,0.162155,-0.018075,-0.061220,0.113934
2007,0.068190,-0.111876,0.044321,0.168390,-0.002245,-0.037930,0.081093
2008,0.055876,-0.093793,0.031562,0.204172,0.029388,-0.142918,0.188517
2009,0.090154,-0.075996,0.028261,0.218144,0.035838,-0.112637,0.219696
2010,0.072757,-0.054805,0.013523,0.269750,0.061702,-0.090813,0.257698


In [30]:
pd.pivot_table(df3.loc[pd.IndexSlice[['AT', 'CZ', 'DK', 'DE', 'SK', 'PL', 'UK'],2002:2018,'bioenergy'], :], index='year', columns='country', values='dependency').to_csv('dependency_2002_2018_bioenergy_selected.csv', decimal=',')


In [31]:
pd.pivot_table(df3.loc[pd.IndexSlice[['AT', 'CZ', 'DK', 'DE', 'SK', 'PL', 'UK'],2002:2018,'solid_biomass'], :], index='year', columns='country', values='dependency').to_csv('dependency_2002_2018_solid_selected.csv', decimal=',')

primary_production      imports      exports  \
country year fuel                                                          
EE      2002 bioenergy              567.283020     0.096000    17.285000   
             biofuels                 0.000000     0.000000     0.000000   
             biogas                   2.078000     0.000000     0.000000   
             ren_mun_waste            0.000000     0.000000     0.000000   
             renewables             567.885010     0.000000    17.030001   
...                                        ...          ...          ...   
        2018 biogas                  13.638000     0.000000     0.000000   
             ren_mun_waste           23.096001     7.739000     0.000000   
             renewables            1743.371948    39.242001   617.226013   
             solid_biomass         1648.012939    14.355000   617.226013   
             total                 6604.620117  2787.822998  2739.016113   

                            gross_inland_consumption  dependency  
country year fuel                                                 
EE      2002 bioenergy                    547.393982   -0.031402  
             biofuels                       0.000000         NaN  
             biogas                         2.078000    0.000000  
             ren_mun_waste                  0.000000         NaN  
             renewables                   548.156006   -0.031068  
...                                              ...         ...  
        2018 biogas                        13.638000    0.000000  
             ren_mun_waste                 30.834999    0.250981  
             renewables                  1155.762939   -0.500089  
             solid_biomass               1035.515991   -0.582194  
             total                       6297.550781    0.007750  

[119 rows x 5 columns]